<a href="https://colab.research.google.com/github/Balavignesh-25/Agentic-AI-System/blob/main/Agentic_AI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 FINAL Agentic AI Demo (Dynamic Tools + Guardrails)

## 1️⃣ Install Dependencies

In [1]:
!pip install -U langchain-groq langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.8
    Uninstalling langchain-core-1.2.8:
      Successfully uninstalled langchain-core-1.2.8


## 2️⃣ Configure API Key

In [2]:
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [3]:
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def get_secret(key_name: str) -> str:
    """
    Securely fetch secrets from Colab userdata or environment variables.

    This function is crucial for managing sensitive information like API keys.
    It prioritizes Google Colab's built-in 'userdata' for security,
    then falls back to environment variables for local development.
    """
    # Check if the code is currently running within a Google Colab environment.
    # This helps in adapting how secrets are accessed.
    if IN_COLAB:
        # In Colab, secrets are managed securely via the 'userdata' interface.
        # This prevents API keys from being directly exposed in the notebook or version control.
        return userdata.get(key_name)
    # If not in Colab (e.g., running locally or on another cloud platform),
    # attempt to retrieve the secret from system environment variables.
    # This provides flexibility for deployment outside of Colab.
    return os.getenv(key_name)

# Retrieve the Groq API key using the `get_secret` function.
# The key named 'GROQ_API_KEY' is expected to be configured in Colab secrets or as an environment variable.
GROQ_API_KEY = get_secret("GROQ_API_KEY")

# Perform a critical validation step: check if the API key was successfully loaded.
# Without a valid API key, the LLM interactions will fail, so this check is essential.
if not GROQ_API_KEY:
    # If the key is missing, raise an error with clear instructions.
    # This guides the user on how to correctly set up their API key.
    raise EnvironmentError(
        "❌ API Key missing!\n"
        "• In Colab → Add it in Secrets and restart runtime\n"
        "• Locally → Export them as environment variables"
    )

print("✅ API key loaded successfully")

✅ API key loaded successfully


## 3️⃣ Initialize LLM

In [4]:

from langchain_groq import ChatGroq
from langchain_core.tools import tool
import json

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.1-8b-instant",
    temperature=0
)


In [5]:
import json
import re

def safe_json_parse(text: str, fallback: dict):
    """
    Safely extract and parse JSON from LLM output.

    LLMs sometimes produce text that isn't perfectly clean JSON.
    This function tries to be robust by first attempting direct parsing,
    and then by trying to extract a JSON-like substring.
    """
    # If the input text is empty or contains only whitespace, it cannot be valid JSON.
    # In such cases, we immediately return a predefined 'fallback' dictionary.
    if not text or not text.strip():
        return fallback

    # First attempt: Try to parse the entire input `text` as a direct JSON string.
    # This is the most straightforward and efficient method if the LLM output is clean.
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # If direct parsing fails (e.g., due to extra text around the JSON or malformed JSON),
        # we don't return an error yet, but proceed to a more lenient extraction method.
        pass # If direct parse fails, try extracting a JSON block

    # Second attempt: If direct parsing failed, try to find a JSON object embedded within the text.
    # This uses a regular expression to locate the first `{...}` block that looks like a JSON object.
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            # If a potential JSON block is found, try parsing just that extracted substring.
            # The `re.DOTALL` flag allows '.' to match newlines, important for multi-line JSON.
            return json.loads(match.group())
        except json.JSONDecodeError:
            # If parsing the extracted block also fails, it means even the embedded part is malformed.
            # We then fall through to return the fallback.
            pass # If parsing the extracted block fails, return fallback

    # If both direct parsing and embedded JSON extraction attempts fail,
    # return the 'fallback' dictionary to ensure the program doesn't crash
    # and has a predictable default value to work with.
    return fallback

## 4️⃣ Guardrails

In [6]:

HIGH_RISK_TOPICS = [
    "stock price",
    "share price",
    "predict",
    "prediction",
    "after 1 hour",
    "next hour"
]

def guardrail_check(goal: str):
    """
    Checks the user's input 'goal' against a list of high-risk topics.

    This function acts as a safety mechanism (guardrail) to prevent the AI
    from engaging in tasks that are deemed unsafe, unethical, or unreliable,
    such as making financial predictions.
    """
    # Iterate through each predefined high-risk topic.
    # This loop systematically checks if any prohibited phrase is present in the user's request.
    for topic in HIGH_RISK_TOPICS:
        # Convert both the topic and the user's goal to lowercase for case-insensitive matching.
        # This ensures that variations like 'Stock Price' or 'stock price' are caught.
        if topic in goal.lower():
            # If a high-risk topic is found, immediately block the request.
            # Return a dictionary indicating it's blocked, along with a clear reason
            # and suggesting safer, alternative queries.
            return {
                "blocked": True,
                "reason": "Financial market prediction is unreliable and unsafe",
                "safe_alternatives": [
                    "Explain how stock prices generally work",
                    "Discuss investment risk factors",
                    "Show historical trends"
                ]
            }
    # If the loop completes without finding any high-risk topics,
    # the request is considered safe and not blocked.
    return {"blocked": False}

In [7]:
PII_KEYWORDS = [
    "name",
    "email",
    "phone",
    "ssn",
    "social security number",
    "address",
    "date of birth",
    "credit card",
    "account number"
]

def pii_filter(text: str):
    """
    Filters input text for Personally Identifiable Information (PII) keywords.

    This function serves as a guardrail to protect user privacy by detecting
    and flagging common PII terms in the input, preventing the AI from processing
    or generating sensitive data inadvertently.
    """
    # Loop through each keyword defined in the PII_KEYWORDS list.
    # This allows for a comprehensive check against multiple types of personal information.
    for keyword in PII_KEYWORDS:
        # Convert the input `text` to lowercase to ensure the PII check is not case-sensitive.
        # This way, 'Name', 'name', and 'NAME' are all detected.
        if keyword in text.lower():
            # If a PII keyword is found, the function immediately returns a 'blocked' status.
            # It provides the specific keyword detected as the reason and suggests an action
            # to the user, like redacting or sanitizing the input.
            return {
                "blocked": True,
                "reason": f"Potential PII detected: '{keyword}'",
                "action": "Redact or sanitize input"
            }
    # If the loop finishes without finding any of the specified PII keywords,
    # the function returns 'blocked': False, indicating that no PII was detected.
    return {"blocked": False}

### How to use the PII filter:

Now you can integrate `pii_filter` into your `agentic_ai_system` or use it independently. Here's an example of how you can test it:

In [8]:
print(pii_filter("My name is John Doe and my email is john.doe@example.com"))
print(pii_filter("Plan a trip to Paris"))

{'blocked': True, 'reason': "Potential PII detected: 'name'", 'action': 'Redact or sanitize input'}
{'blocked': False}


## 5️⃣ Tools (Dynamic Inputs)

In [9]:
from langchain_core.tools import tool

# 🇮🇳 Domestic Trip Tool
# This tool is specifically designed to handle travel plans within India.
# It's an example of a specialized function that an AI agent might call.
@tool
def within_india_trip_tool(destination: str, days: int, budget: int) -> dict:
    """
    Plan a domestic trip within India.

    Inputs:
    - destination: city or state name
    - days: number of travel days
    - budget: total budget in INR

    Output:
    - Structured domestic travel plan
    """
    # This block defines the specific output structure for a domestic trip.
    # It includes details like hotel type, common assumptions, and potential risks,
    # providing a rich, structured response to the user's request.
    return {
        "destination": destination,
        "duration_days": days,
        "budget_estimate": budget,
        "hotel_type": "3-star",
        "assumptions": [
            "Domestic travel",
            "Economy accommodation"
        ],
        "risks": [
            "Weather dependency",
            "Seasonal crowd"
        ]
    }


# 🌍 International Trip Tool
# This tool handles travel plans to destinations outside India, requiring different considerations.
@tool
def outside_india_trip_tool(destination: str, days: int, budget: int) -> dict:
    """
    Plan an international trip.

    Inputs:
    - destination: country or city name
    - days: number of travel days
    - budget: total budget in INR

    Output:
    - Structured international travel plan
    """
    # This output structure is tailored for international travel, reflecting specific factors
    # like visa requirements and currency fluctuations that are relevant globally.
    return {
        "destination": destination,
        "duration_days": days,
        "budget_estimate": budget,
        "hotel_type": "4-star",
        "assumptions": [
            "Visa approved",
            "International flights available"
        ],
        "risks": [
            "Visa delay",
            "Currency fluctuation"
        ]
    }


# 🧑‍💻 College Hackathon Tool
# A tool for organizing hackathons specifically tailored for a college environment.
@tool
def college_hackathon_tool(theme: str, team_size: int) -> dict:
    """
    Organize a college-level hackathon.

    Inputs:
    - theme: hackathon theme
    - team_size: members per team

    Output:
    - College hackathon configuration
    """
    # The output specifies parameters relevant to a college hackathon, such as a typical duration.
    return {
        "type": "College Hackathon",
        "theme": theme,
        "team_size": team_size,
        "duration_hours": 36
    }


# 🏢 Corporate Hackathon Tool
# This tool is for organizing hackathons in a corporate setting, which may have different requirements.
@tool
def corporate_hackathon_tool(theme: str, team_size: int) -> dict:
    """
    Organize a corporate-level hackathon.

    Inputs:
    - theme: hackathon theme
    - team_size: members per team

    Output:
    - Corporate hackathon configuration
    """
    # The output reflects corporate hackathon specifics, like a potentially longer duration.
    return {
        "type": "Corporate Hackathon",
        "theme": theme,
        "team_size": team_size,
        "duration_hours": 48
    }


# 🎉 Small Event Tool
# A tool to plan smaller, private social events.
@tool
def small_event_tool(event_type: str, guests: int) -> dict:
    """
    Plan a small private event.

    Inputs:
    - event_type: birthday, family meet, get-together
    - guests: number of attendees

    Output:
    - Small event planning details
    """
    # Details specific to small events, often including simpler activity suggestions.
    return {
        "event_type": event_type,
        "guests": guests,
        "activities": [
            "Music",
            "Games",
            "Food"
        ]
    }


# 🎪 Large Event Tool
# This tool is for planning large-scale public events, with more complex requirements.
@tool
def large_event_tool(event_type: str, guests: int) -> dict:
    """
    Plan a large public event.

    Inputs:
    - event_type: concert, festival, celebration
    - guests: number of attendees

    Output:
    - Large-scale event planning details
    """
    # Output tailored for large events, which involve complex logistics like security and catering.
    return {
        "event_type": event_type,
        "guests": guests,
        "activities": [
            "Stage Show",
            "Catering",
            "Security"
        ]
    }

## 6️⃣ Brain Agent

In [10]:

def brain_decide_agent(goal: str):
    """
    Acts as the primary router, deciding which specialized agent should handle the user's request.

    This function is critical for the agentic system's modularity, ensuring that
    each request is directed to the most appropriate backend tool or agent.
    It uses the LLM to categorize the user's intent.
    """
    # Construct a clear and concise prompt for the LLM.
    # The prompt instructs the LLM to act as a decision-making brain and to output
    # its decision in a strict JSON format, selecting one of the predefined agents.
    prompt = f"""
You are a decision-making brain.

Respond with ONLY valid JSON.
No explanation text.

Choose one agent:
- vacation
- hackathon
- social_event
- unknown

JSON format:
{{
  "agent": "vacation | hackathon | social_event | unknown",
  "confidence": 0.0,
  "reason": "short reason"
}}

User goal: "{goal}"
"""

    # Invoke the LLM (Large Language Model) with the crafted prompt.
    # The LLM processes the user's goal and returns its decision as raw text.
    raw = llm.invoke(prompt).content

    # Safely parse the raw LLM output into a structured JSON object.
    # The `safe_json_parse` function is used here to handle potential formatting
    # issues in the LLM's response, providing a robust parsing mechanism.
    # A `fallback` dictionary is provided to ensure a default decision if parsing fails.
    return safe_json_parse(
        raw,
        fallback={
            "agent": "unknown",
            "confidence": 0.0,
            "reason": "Invalid or empty model response"
        }
    )


## 7️⃣ Vacation Agent

In [11]:
def vacation_agent(goal: str):
    """
    Specialized agent for planning vacation trips.

    It extracts key details like destination, duration, and budget from the user's request
    and then routes the request to either a 'within_india' or 'outside_india' trip planning tool.
    """
    print("\n🧠 Vacation agent reasoning...")

    # Formulate a specific prompt to instruct the LLM to act as an information extraction system.
    # The prompt guides the LLM to identify critical parameters for vacation planning
    # (destination, days, budget) and to decide between domestic or international tools.
    prompt = f"""
You are an information extraction system.

Extract the following from the user request:
- destination (string)
- number of days (integer)
- budget in INR (integer)

Rules:
- Do NOT guess
- If a value is missing, infer reasonably
- Explain clearly WHY the chosen tool is appropriate
- Assign a realistic confidence score (0.5–0.95)
- Return ONLY valid JSON
- NO explanation text

JSON format:
{{
  "tool": "within_india | outside_india",
  "destination": "",
  "days": 0,
  "budget": 0,
  "tool_confidence": 0.0,
  "reason": "Why this tool is appropriate"
}}

User request:
"{goal}"
"""

    # Send the extraction prompt to the LLM and capture its raw text output.
    raw = llm.invoke(prompt).content
    print("🔍 Raw LLM extraction:", raw)

    # Parse the LLM's raw output into a structured decision dictionary.
    # The `safe_json_parse` function ensures robustness against malformed LLM responses.
    # A default fallback is provided if extraction or parsing fails, ensuring continuity.
    decision = safe_json_parse(
        raw,
        {
            "tool": "within_india",
            "destination": "Unknown",
            "days": 3,
            "budget": 30000,
            "reason": "Fallback values used"
        }
    )

    # 🔐 HARD VALIDATION (IMPORTANT): Ensure 'days' is an integer.
    # This step converts the extracted 'days' value to an integer, as the tool expects it.
    # It's a crucial type enforcement for robust tool invocation.
    if not isinstance(decision.get("days"), int):
        decision["days"] = int(decision["days"])

    # 🔐 HARD VALIDATION (IMPORTANT): Ensure 'budget' is an integer.
    # Similarly, the 'budget' is cast to an integer to match the tool's expected input type.
    if not isinstance(decision.get("budget"), int):
        decision["budget"] = int(decision["budget"])

    print("✅ Parsed decision:", decision)

    # Based on the 'tool' determined by the LLM (either 'within_india' or 'outside_india'),
    # the appropriate trip planning tool is invoked with the extracted parameters.
    if decision["tool"] == "within_india":
        result = within_india_trip_tool.invoke({
            "destination": decision["destination"],
            "days": decision["days"],
            "budget": decision["budget"]
        })
    else:
        result = outside_india_trip_tool.invoke({
            "destination": decision["destination"],
            "days": decision["days"],
            "budget": decision["budget"]
        })

    # Finally, return a structured result from the vacation agent, summarizing
    # which agent and tool were used, the reasoning, and the plan generated by the tool.
    return {
        "agent": "Vacation Organizer",
        "tool_used": decision["tool"],
        "reason": decision["reason"],
        "result": result
    }

## 8️⃣ Hackathon Agent

In [12]:
def hackathon_agent(goal: str):
    """
    Specialized agent for organizing hackathons.

    It extracts the hackathon type (college/corporate), theme, and team size
    from the user's request and invokes the corresponding hackathon planning tool.
    """
    print("\n🧠 Hackathon agent reasoning...")

    # Define the prompt for the LLM, instructing it to extract specific details
    # related to hackathon organization, such as type, theme, and team size.
    # The LLM is also guided to output its findings in a consistent JSON format.
    prompt = f"""
You are an information extraction system.

Extract the following from the user request:
- hackathon type: college or corporate
- theme (string)
- team size (integer)

Rules:
- Do NOT guess wildly
- Infer reasonably if missing
- Return ONLY valid JSON
- NO explanation text

JSON format:
{{
  "tool": "college | corporate",
  "theme": "",
  "team_size": 0,
  "tool_confidence": 0.0,
  "reason": "Why this tool is appropriate"
}}

User request:
"{goal}"
"""

    # Invoke the LLM with the prompt to perform the information extraction.
    # The raw text response from the LLM is captured for subsequent parsing.
    raw = llm.invoke(prompt).content
    print("🔍 Raw LLM extraction:", raw)

    # Safely parse the raw LLM output into a structured dictionary.
    # `safe_json_parse` handles potential parsing errors, and a fallback provides defaults.
    decision = safe_json_parse(
        raw,
        {
            "tool": "college",
            "theme": "General Technology",
            "team_size": 4,
            "reason": "Fallback values used"
        }
    )

    # 🔐 HARD VALIDATION: Ensure 'team_size' is an integer.
    # This is important to ensure the tool receives numerical input as expected.
    if not isinstance(decision.get("team_size"), int):
        decision["team_size"] = int(decision["team_size"])

    print("✅ Parsed decision:", decision)

    # Route the request to either the 'college_hackathon_tool' or 'corporate_hackathon_tool'
    # based on the LLM's classification of the hackathon type.
    if decision["tool"] == "college":
        result = college_hackathon_tool.invoke({
            "theme": decision["theme"],
            "team_size": decision["team_size"]
        })
    else:
        result = corporate_hackathon_tool.invoke({
            "theme": decision["theme"],
            "team_size": decision["team_size"]
        })

    # Return a comprehensive result from the hackathon agent, detailing the chosen tool
    # and the structured output from that tool.
    return {
        "agent": "Hackathon Organizer",
        "tool_used": decision["tool"],
        "reason": decision["reason"],
        "result": result
    }

## 9️⃣ Social Event Agent

In [13]:
def social_event_agent(goal: str):
    """
    Specialized agent for planning social events.

    It extracts the event type and number of guests from the user's request
    and categorizes the event as 'small' or 'large' to invoke the appropriate tool.
    """
    print("\n🧠 Social Event agent reasoning...")

    # Create a prompt for the LLM to extract key information for social event planning.
    # This includes the event type, number of guests, and the inferred scale of the event.
    prompt = f"""
You are an information extraction system.

Extract the following from the user request:
- event type (string)
- number of guests (integer)
- event scale: small or large

Rules:
- Do NOT guess wildly
- Infer reasonably if missing
- Return ONLY valid JSON
- NO explanation text

JSON format:
{{
  "tool": "small | large",
  "event_type": "",
  "guests": 0,
 "tool_confidence": 0.0,
  "reason": "Why this tool is appropriate"
}}

User request:
"{goal}"
"""

    # Execute the LLM with the extraction prompt and retrieve the raw response.
    raw = llm.invoke(prompt).content
    print("🔍 Raw LLM extraction:", raw)

    # Parse the LLM's raw output into a structured decision, using `safe_json_parse`
    # to handle potential inconsistencies and provide default values if necessary.
    decision = safe_json_parse(
        raw,
        {
            "tool": "small",
            "event_type": "General Event",
            "guests": 30,
            "reason": "Fallback values used"
        }
    )

    # 🔐 HARD VALIDATION: Ensure 'guests' is an integer.
    # This step converts the guest count to an integer to ensure correct input for the tools.
    if not isinstance(decision.get("guests"), int):
        decision["guests"] = int(decision["guests"])

    print("✅ Parsed decision:", decision)

    # Based on whether the LLM classified the event as 'small' or 'large',
    # the appropriate social event planning tool is called.
    if decision["tool"] == "small":
        result = small_event_tool.invoke({
            "event_type": decision["event_type"],
            "guests": decision["guests"]
        })
    else:
        result = large_event_tool.invoke({
            "event_type": decision["event_type"],
            "guests": decision["guests"]
        })

    # Return a comprehensive summary of the social event planning, including
    # the agent, tool used, reasoning, and the structured event plan.
    return {
        "agent": "Social Event Organizer",
        "tool_used": decision["tool"],
        "reason": decision["reason"],
        "result": result
    }

 🔟 Master Agent Controller

In [14]:
def agentic_ai_system(goal: str):
    """
    The master controller of the agentic AI system.

    This function orchestrates the entire process: from initial guardrail checks
    to routing the request to the correct specialized agent (vacation, hackathon, or social event).
    It ensures safety and proper delegation of tasks.
    """
    # Step 1: Initial Guardrail Check.
    # Before any processing, the user's input 'goal' is checked against defined safety policies.
    # This is a critical first line of defense against inappropriate or unsafe requests.
    guard = guardrail_check(goal)
    if guard["blocked"]:
        # If the guardrail detects a violation, immediately return the blocking response.
        # No further processing occurs, ensuring the system adheres to safety guidelines.
        return guard

    # Step 2: Brain Agent Decision.
    # If the goal passes the guardrail, the `brain_decide_agent` is invoked.
    # This agent acts as a router, determining which specialized agent (e.g., vacation, hackathon)
    # is best suited to handle the user's specific request.
    brain = brain_decide_agent(goal)

    # Step 3: Route to Specialized Agent.
    # Based on the decision from the `brain` agent, the request is dispatched
    # to the appropriate specialized agent for detailed processing.
    if brain["agent"] == "vacation":
        return vacation_agent(goal)
    elif brain["agent"] == "hackathon":
        return hackathon_agent(goal)
    elif brain["agent"] == "social_event":
        return social_event_agent(goal)
    else:
        # If the brain agent cannot confidently determine a suitable specialized agent,
        # an "uncertain" status is returned. This indicates that the system either
        # does not have a tool for this request or could not understand the intent.
        return {"status": "uncertain", "message": "Request cannot be handled reliably"}

## ▶️ Demo Runs

In [15]:
agentic_ai_system("Plan a 10 day trip to Forest anywhere with 40k budget suggest")


🧠 Vacation agent reasoning...
🔍 Raw LLM extraction: {
  "tool": "within_india",
  "destination": "Forest",
  "days": 10,
  "budget": 40000,
  "tool_confidence": 0.8,
  "reason": "The user mentioned 'Forest' which is a common name for a forest in India, suggesting a domestic trip."
}
✅ Parsed decision: {'tool': 'within_india', 'destination': 'Forest', 'days': 10, 'budget': 40000, 'tool_confidence': 0.8, 'reason': "The user mentioned 'Forest' which is a common name for a forest in India, suggesting a domestic trip."}


{'agent': 'Vacation Organizer',
 'tool_used': 'within_india',
 'reason': "The user mentioned 'Forest' which is a common name for a forest in India, suggesting a domestic trip.",
 'result': {'destination': 'Forest',
  'duration_days': 10,
  'budget_estimate': 40000,
  'hotel_type': '3-star',
  'assumptions': ['Domestic travel', 'Economy accommodation'],
  'risks': ['Weather dependency', 'Seasonal crowd']}}

In [16]:
agentic_ai_system("Organize a all india hackathon finals in delhi")


🧠 Hackathon agent reasoning...
🔍 Raw LLM extraction: {
  "tool": "college",
  "theme": "all india hackathon finals in delhi",
  "team_size": 0,
  "tool_confidence": 1.0,
  "reason": "The term 'all india' suggests a national scope, which is often associated with college-level hackathons."
}
✅ Parsed decision: {'tool': 'college', 'theme': 'all india hackathon finals in delhi', 'team_size': 0, 'tool_confidence': 1.0, 'reason': "The term 'all india' suggests a national scope, which is often associated with college-level hackathons."}


{'agent': 'Hackathon Organizer',
 'tool_used': 'college',
 'reason': "The term 'all india' suggests a national scope, which is often associated with college-level hackathons.",
 'result': {'type': 'College Hackathon',
  'theme': 'all india hackathon finals in delhi',
  'team_size': 0,
  'duration_hours': 36}}

In [17]:
agentic_ai_system("Plan a politics party for 1000 guests")


🧠 Social Event agent reasoning...
🔍 Raw LLM extraction: {
  "tool": "large",
  "event_type": "politics party",
  "guests": 1000,
  "tool_confidence": 1.0,
  "reason": "The event type is politics party and the number of guests is 1000, which is a large scale event."
}
✅ Parsed decision: {'tool': 'large', 'event_type': 'politics party', 'guests': 1000, 'tool_confidence': 1.0, 'reason': 'The event type is politics party and the number of guests is 1000, which is a large scale event.'}


{'agent': 'Social Event Organizer',
 'tool_used': 'large',
 'reason': 'The event type is politics party and the number of guests is 1000, which is a large scale event.',
 'result': {'event_type': 'politics party',
  'guests': 1000,
  'activities': ['Stage Show', 'Catering', 'Security']}}

In [18]:
agentic_ai_system("Stock market price next min predict")

{'blocked': True,
 'reason': 'Financial market prediction is unreliable and unsafe',
 'safe_alternatives': ['Explain how stock prices generally work',
  'Discuss investment risk factors',
  'Show historical trends']}